# FIT5196 Assessment 1 Task 2
#### Student Name: Dawei Gu
#### Student ID: 29910226
Tutorial Section: FIT5196 Tutorial 12

Tutor: Islam Nassar

Start Date: 29/03/2017

Last Modified Date: 04/04/2017

Environment: Python 3.7 and Anaconda 2018.12 (64-bit)

Libraries used: pdfplumber, pandas, nltk

In [1]:
import pdfplumber
import pandas as pd
import nltk.data
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.util import ngrams
from nltk.probability import *

Using pdfplumber package for reading the table from "29910226.pdf", and convert the table of each pages into dataframe, and extract unit data from dataframe of each pages unit by unit. Unit data will be ordered list which include unit_id, synopsis and outcome. 

In [2]:
raw_data = []
pdf = pdfplumber.open("29910226.pdf")
for page in pdf.pages:
    df = pd.DataFrame(page.extract_tables())
    for i in range(df.shape[1]):
        raw_data.append(df.iloc[0,i])
raw_data.pop(0)

['Title', 'Synopsis', 'Outcomes']

### Task 2 Part 1 
##### Processing data with 7 requirement

The function lower__case meets the Requirement 5: Tokens must be normalized to lowercase except the capital tokens appeared in the middle of a sentence/line.
- First function separete the string of synopsis and outcome into sentence.
- Using sent_detector from nltk for synopsis to gerenate the list of sentence from synopsis.
- Since outcome is string of list of sentence, split outcome into sentence by ";"， remove quotation mark and comma between quotation marks. 
- Put all of sentence together as order from synopsis and outcome, using a for loop to make the first letter of sentence lower case
- Put all processed sentence into one string for unit as return result.

In [3]:
def lower_case(unit):
    lower_string = "" 
    string_list = []
    synopsis = unit[1].replace("\n", " ")
    synopsis = sent_detector.tokenize(synopsis)
    outcome = unit[2][1:-1].replace("\n", " ").split(";")  # remove the "[" and "]" at begining and end of outcome string, split them base on ";"
    outcome = [item.replace("\',", "").replace("\",", "").replace("\'", "").replace("\"", "") for item in outcome] # remove the"," and talking mark
    sent_list = synopsis + outcome # join all of sentence from synopsis and outcome into 1 list
    for item in sent_list:
        if len(item) > 0:
            string_temp = item[0].lower() + item[1:]
            string_list.append(string_temp)
#     string_list = [item[0].lower() + item[1:] for item in sent_list] # lower case the fisrt letter of sentence inside of list
    lower_string = " ".join(string_list) # join all of sentence into one string. 
    return lower_string
# print(lower_case(raw_data[0])) # For testing

Load the stop words from stopwords_en.txt

In [4]:
file_handle = open("stopwords_en.txt", "r")
stop_words = []
for line in file_handle:
    stop_words.append(line.strip())
file_handle.close
# print(stop_words)

<function TextIOWrapper.close()>

The function process_token to meet following requirement:<br>
Requirement 1: The word tokenization must use the following regular expression, "\w+(?:[-']\w+)?"<br>
Requirement 2 (Partly): The context-independent and context-dependent (with the threshold set to %95) stop words must be removed from the vocab. The stop words list (i.e, stopwords_en.txt ) provided in the zip file must be used. <br>
Requriement 3: Tokens should be stemmed using the Porter stemmer.<br>
Requirement 5(done by function lower_case): Tokens must be normalized to lowercase except the capital tokens appeared in the middle of a sentence/line.<br>
Requriement 6: Tokens with the length less than 3 should be removed from the vocab.

- Lower_case fucntion to done the normalization for requriement 5. unit_string is whole piece of string for that unit.
- Using regexptokenizer with regular expression "\w+(?:[-']\w+)?" to extract the tokens from string of unit. This step meets requirment 1.
- Filtering the tokens against stop words list. This step meets part of requirement 2.
- Filtering the tokens with length less than 3 out (ps. tokens with length of 3 include, will not be filtered out), this step meet requirement 6
- Using PorterStemmer from nltk to stem tokens. this step meet requirement 3.
- Make another filtering length of tokens, because stemmer may make the length of some tokens less than 3.
- Return the tokens list of that unit

In [5]:
def process_tokens(unit):
    unit_string = lower_case(unit)
    tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?") 
    pre_tokens = tokenizer.tokenize(unit_string)
    no_stop_tokens = [token for token in pre_tokens if token not in stop_words and len(token) >= 3]
    stemmed_tokens = [stemmer.stem(token) for token in no_stop_tokens]
    tokens = [token for token in stemmed_tokens if len(token) >= 3]
    return tokens
# print(process_tokens(raw_data[0]))   # For testing

Define a function name formation_data used for make data information into a basic data form for further processing. Using a for loop to run that function to all of units.  

In [6]:
def formation_data(unit):
    data_result = {}
    data_result["id"] = unit[0]
    data_result["pre_token"] = process_tokens(unit)
    return data_result


pre_data = []
for unit in raw_data:
    pre_data.append(formation_data(unit))
for item in pre_data:
    print(item)


{'id': 'ATS3503', 'pre_token': ['unit', 'examin', "women'", 'literatur', 'rang', 'period', 'countri', 'issu', 'examin', 'includ', 'place', "women'", 'write', 'literari', 'canon', 'literari', 'histori', 'strain', 'feminist', 'literari', 'critic', 'theori', 'recent', 'debat', 'recept', "women'", 'write', 'literatur', 'make', 'intervent', 'social', 'norm', 'gender', 'sexual', 'typic', 'theme', "women'", 'write', 'marriag', 'motherhood', 'work', 'violenc', 'abil', 'assess', 'interpret', 'relationship', 'literari', 'text', 'social', 'polit', 'context', 'inform', 'understand', 'key', 'approach', 'studi', 'women', 'write', 'develop', 'inform', 'critic', 'perspect', 'question', 'gender', 'literari', 'product', 'literari', 'solid', 'advanc', 'undergraduate-level', 'skill', 'read', 'interpret', 'literari', 'text']}
{'id': 'ATS1304', 'pre_token': ['employ', 'textual', 'critic', 'critic', 'theori', 'theori', 'polit', 'economi', 'unit', 'introduc', 'varieti', 'approach', 'studi', 'televis', 'phenom

Following code is used to achevie following:<br>
Requirment 4 Rare tokens (with the threshold set to %5) must be removed from the vocab.<br>
Requirement 2 (Partly): context-dependent (with the threshold set to %95) stop words must be removed from the vocab<br>

- make list of each unit tokens into set to get unique tokens of each unit. and make the set of unique tokens back to list. This step is used to make count for tokens appeared. For example, if there certain token appear 3 times in first unit, there will be only one that token appear in the first unit's tokens list after this step, it only mean it appear in that unit.
- Put all of unit tokens after step 1 into one list, and using freqdist to count the frequency of tokens. For example, count value of certain token is 15, it means that token appear in 15 unit datas.
- Calculate the upper limit which is 190 (95% of 200 units)(ps.190 is not include), if the certain tokens appear in more than 95% of total unit data, they will be count as context-dependent stop words, they should not be included inside of vocabulary. This one meets part of requriement 2.
- Calculate the lower limit which is 10 (5% of 200 units)(ps.10 is not include), if the certain toekns appear less than 5% of total unit data, they will be count as rare tokens, they also should not be included inside of vocabulary. This one meets requriement 4. 
- Vocabulary list sort.  

In [7]:
total_pre_token = []
for item in pre_data:
    total_pre_token.extend(list(set(item["pre_token"])))
print(len(total_pre_token))
print(len(set(total_pre_token)))

freq_token = FreqDist(total_pre_token)
upper_limit = int(200*0.95)
lower_limit = int(200*0.05)
print("upper_limit:", upper_limit, ",lower_limit:", lower_limit)


voc = []
for word, count in freq_token.items():
    if count >= lower_limit and count <= upper_limit:
        voc.append(word)
#         print(word, count)
voc.sort()
print(len(voc))
print(voc)

12718
2563
upper_limit: 190 ,lower_limit: 10
300
['abil', 'academ', 'account', 'acquir', 'activ', 'address', 'advanc', 'aim', 'analys', 'analysi', 'analyt', 'appli', 'applic', 'apprais', 'appreci', 'approach', 'area', 'argument', 'articul', 'aspect', 'assess', 'awar', 'base', 'basic', 'behaviour', 'biolog', 'broad', 'build', 'busi', 'capabl', 'care', 'case', 'challeng', 'chang', 'clinic', 'cognit', 'collabor', 'collect', 'combin', 'common', 'commun', 'compar', 'compet', 'complet', 'complex', 'compon', 'comput', 'concept', 'conceptu', 'condit', 'conduct', 'conflict', 'consid', 'construct', 'contemporari', 'context', 'continu', 'contribut', 'control', 'core', 'cover', 'creat', 'creativ', 'critic', 'critiqu', 'cultur', 'current', 'data', 'debat', 'decis', 'defend', 'defin', 'deliveri', 'demonstr', 'describ', 'design', 'detail', 'determin', 'develop', 'differ', 'disciplin', 'discuss', 'diseas', 'disord', 'divers', 'document', 'econom', 'effect', 'element', 'emerg', 'emphasi', 'employ', 'en

Define a function to make bigram into string form like "token1_token2"

In [8]:
def bigram_convert(bigram):
    result = ""
    result = str(bigram[0]) + "_" + str(bigram[1])
    return result

Make function final_process to filter out the rare tokens and context dependent stop words, and also generate the bigrams (using bigram_convert function to convert into string). And for loop is used for run through all of unit.  

In [9]:
def final_process(unit):
    result = {}
    bigram_unit_temp = []
    result["id"] = unit["id"]
    result["token"] = [token for token in unit["pre_token"] if token in voc]
    bigram_unit_temp = list(ngrams(result["token"], n = 2))
    result["bigram"] = [bigram_convert(bigram) for bigram in bigram_unit_temp]
    return result
data = []
for unit in pre_data:
    data.append(final_process(unit))
# print(data[0])  # For testing

### Task 2 Part 2 
##### Making vocabulary and vector space model

Put all of token from each units together to generate the top 200 bigrams by pmi, and using bigram_convert function to convert into certain form string, sort the bigram_list in the end. This step meets following requirement: <br>
Requirement 7. First 200 meaningful bigrams (i.e., collocations) must be included in the vocab using PMI measure.

In [10]:
total_token = []
for item in data:
    total_token.extend(item["token"])
print(len(total_token))
print(len(set(total_token)))

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(total_token)
bigram_list = finder.nbest(bigram_measures.pmi, 200)
bigram_list = [bigram_convert(bigram) for bigram in bigram_list]
bigram_list.sort()
print(len(bigram_list))
print(bigram_list)

12395
300
200
['acquir_conceptu', 'acquir_prepar', 'address_conflict', 'address_question', 'aim_equip', 'analyt_represent', 'analyt_tool', 'apprais_type', 'appreci_incorpor', 'awar_local', 'base_intervent', 'behaviour_combin', 'behaviour_real', 'biolog_diseas', 'biolog_factor', 'broad_detail', 'broad_rang', 'capabl_defend', 'capabl_detail', 'case_studi', 'chang_conflict', 'chang_deliveri', 'chang_local', 'cognit_behaviour', 'cognit_biolog', 'cognit_creativ', 'cognit_disord', 'cognit_enhanc', 'collabor_defend', 'collect_analysi', 'combin_element', 'combin_power', 'compar_perspect', 'complet_cognit', 'complet_enabl', 'complet_independ', 'compon_compon', 'comput_implement', 'comput_model', 'comput_softwar', 'conceptu_broad', 'conceptu_overview', 'condit_real', 'conduct_independ', 'conflict_object', 'consid_detail', 'consid_emphasi', 'consid_underli', 'continu_chang', 'continu_familiar', 'continu_overview', 'control_conflict', 'core_valu', 'creat_combin', 'creat_innov', 'creat_success', 'c

- Adding top 200 bigrams into vocabulary list.
- Using enumerate function to generate the vocabulary with its index
- Write the vocabulary with ites index into file named "29910226.txt".

In [11]:
# Adding bigram into vocabulary list
voc.extend(bigram_list)
print(len(voc))

# Gerenrate the vocabulary list with index of each vocabulary
voc_index = list(enumerate(voc))
print(voc_index)

# Write the information of each vocabulary into file
file_handle = open("29910226_vocab.txt", "w")
for item in voc_index:
    file_handle.write(str(item[1]) + ":" + str(item[0]) + "\n")
file_handle.close()

500
[(0, 'abil'), (1, 'academ'), (2, 'account'), (3, 'acquir'), (4, 'activ'), (5, 'address'), (6, 'advanc'), (7, 'aim'), (8, 'analys'), (9, 'analysi'), (10, 'analyt'), (11, 'appli'), (12, 'applic'), (13, 'apprais'), (14, 'appreci'), (15, 'approach'), (16, 'area'), (17, 'argument'), (18, 'articul'), (19, 'aspect'), (20, 'assess'), (21, 'awar'), (22, 'base'), (23, 'basic'), (24, 'behaviour'), (25, 'biolog'), (26, 'broad'), (27, 'build'), (28, 'busi'), (29, 'capabl'), (30, 'care'), (31, 'case'), (32, 'challeng'), (33, 'chang'), (34, 'clinic'), (35, 'cognit'), (36, 'collabor'), (37, 'collect'), (38, 'combin'), (39, 'common'), (40, 'commun'), (41, 'compar'), (42, 'compet'), (43, 'complet'), (44, 'complex'), (45, 'compon'), (46, 'comput'), (47, 'concept'), (48, 'conceptu'), (49, 'condit'), (50, 'conduct'), (51, 'conflict'), (52, 'consid'), (53, 'construct'), (54, 'contemporari'), (55, 'context'), (56, 'continu'), (57, 'contribut'), (58, 'control'), (59, 'core'), (60, 'cover'), (61, 'creat'),

Define a function to make count vector. 

- Add first item which is id of unit into result list.
- Using freqdist to count the token of one unit.
- Extract keys of freqdist generated dictionary
- Make a for loop for check each item of voc_index with order of voc_index, compare the each vocabulary with keys from unit token freqdist.
- If there match, makeing form as "match_vocabulary_index : match_keys_token_value(count)"
- append string_temp as order of voc_index into result which is a list.
- Join all of items inside of result list as one string. 


In [12]:
def count_vect(unit, voc_index):
    unit_result = [str(unit["id"])]
    token_unit_total = []
    token_unit_total.extend(unit["bigram"])
    token_unit_total.extend(unit["token"])
    token_unit_dict = dict(FreqDist(token_unit_total))
    token_unit_key = list(token_unit_dict.keys())
    for item in voc_index:
        if item[1] in token_unit_key:
            string_temp = str(item[0]) + ":" + str(token_unit_dict[item[1]])
            unit_result.append(string_temp)
#             if item[0] > 300:   # used for check bigram match
#                 print("yes")
    unit_result = ",".join(unit_result)
    return unit_result


Run all of unit data through count_vect function

In [13]:
unit_count_vect = []
for unit in data:
    unit_count_vect.append(count_vect(unit, voc_index))
print(len(unit_count_vect))
print(unit_count_vect[0]) # For testing

200
ATS3503,0:1,6:1,15:1,20:1,55:1,63:2,68:1,78:1,101:2,126:1,135:1,141:2,147:2,148:1,153:1,155:1,165:2,168:1,195:1,197:1,200:1,210:1,222:1,223:1,224:1,231:1,252:1,253:2,266:1,278:2,279:1,281:1,289:1,291:1,296:1,298:4


Write vector space model into file named "29910226.txt"

In [14]:
file_handle = open("29910226_countVec.txt", "w")
for item in unit_count_vect:
    file_handle.write(str(item) + "\n")
file_handle.close()